In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib
# pickle, hd5 is another python library that does the same

In [1]:
import pandas as pd
import tensorflow
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical 

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head(10)

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
5,CONFIRMED,0,0,0,0,2.566589,1.780000e-05,-1.780000e-05,179.554370,0.004610,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,CONFIRMED,0,0,0,0,16.068647,1.090000e-05,-1.090000e-05,173.621937,0.000517,...,-83,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841
7,CONFIRMED,0,0,0,0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,...,-78,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,49.316399,11.338
8,CONFIRMED,0,1,0,0,2.204735,4.300000e-08,-4.300000e-08,121.358542,0.000016,...,-89,4.019,0.033,-0.027,1.952,0.099,-0.110,292.24728,47.969521,10.463
9,CONFIRMED,0,0,0,0,3.522498,1.980000e-07,-1.980000e-07,121.119423,0.000047,...,-137,4.169,0.055,-0.045,1.451,0.110,-0.110,281.28812,42.451080,13.563


In [3]:
df.describe()


,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,...,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,0.157059,0.244743,0.202975,0.125018,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,...,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,0.363882,0.429966,0.402243,0.330763,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,...,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,...,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,...,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,0.000000,0.000000,0.000000,0.000000,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,...,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,...,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,...,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


In [4]:
df.columns


Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [5]:
df.isnull().sum()

koi_disposition      0
koi_fpflag_nt        0
koi_fpflag_ss        0
koi_fpflag_co        0
koi_fpflag_ec        0
koi_period           0
koi_period_err1      0
koi_period_err2      0
koi_time0bk          0
koi_time0bk_err1     0
koi_time0bk_err2     0
koi_impact           0
koi_impact_err1      0
koi_impact_err2      0
koi_duration         0
koi_duration_err1    0
koi_duration_err2    0
koi_depth            0
koi_depth_err1       0
koi_depth_err2       0
koi_prad             0
koi_prad_err1        0
koi_prad_err2        0
koi_teq              0
koi_insol            0
koi_insol_err1       0
koi_insol_err2       0
koi_model_snr        0
koi_tce_plnt_num     0
koi_steff            0
koi_steff_err1       0
koi_steff_err2       0
koi_slogg            0
koi_slogg_err1       0
koi_slogg_err2       0
koi_srad             0
koi_srad_err1        0
koi_srad_err2        0
ra                   0
dec                  0
koi_kepmag           0
dtype: int64

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_co',
         'koi_period', 'koi_period_err1', 
       'koi_time0bk', 'koi_time0bk_err1', 'koi_impact',
       'koi_impact_err1', 'koi_duration',
       'koi_duration_err1', 'koi_depth', 'koi_depth_err1',
       'koi_prad', 'koi_prad_err1', 
       'koi_teq', 'koi_insol', 'koi_insol_err1', 
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
        'koi_slogg', 'koi_slogg_err1', 
       'koi_srad', 'koi_srad_err1', 'ra', 'dec',
       'koi_kepmag']]
selected_features

,koi_fpflag_co,koi_period,koi_period_err1,koi_time0bk,koi_time0bk_err1,koi_impact,koi_impact_err1,koi_duration,koi_duration_err1,koi_depth,...,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_slogg,koi_slogg_err1,koi_srad,koi_srad_err1,ra,dec,koi_kepmag
0,0,54.418383,2.479000e-04,162.513840,0.003520,0.586,0.059,4.50700,0.11600,874.8,...,2,5455,81,4.467,0.064,0.927,0.105,291.93423,48.141651,15.347
1,0,19.899140,1.490000e-05,175.850252,0.000581,0.969,5.126,1.78220,0.03410,10829.0,...,1,5853,158,4.544,0.044,0.868,0.233,297.00482,48.134129,15.436
2,0,1.736952,2.630000e-07,170.307565,0.000115,1.276,0.115,2.40641,0.00537,8079.2,...,1,5805,157,4.564,0.053,0.791,0.201,285.53461,48.285210,15.597
3,0,2.525592,3.760000e-06,171.595550,0.001130,0.701,0.235,1.65450,0.04200,603.3,...,1,6031,169,4.438,0.070,1.046,0.334,288.75488,48.226200,15.509
4,0,4.134435,1.050000e-05,172.979370,0.001900,0.762,0.139,3.14020,0.06730,686.0,...,2,6046,189,4.486,0.054,0.972,0.315,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,8.589871,1.846000e-04,132.016100,0.015700,0.765,0.023,4.80600,0.63400,87.7,...,1,5638,169,4.296,0.231,1.088,0.313,298.74921,46.973351,14.478
6987,1,0.527699,1.160000e-07,131.705093,0.000170,1.252,0.051,3.22210,0.01740,1579.2,...,1,5638,139,4.529,0.035,0.903,0.237,297.18875,47.093819,14.082
6988,0,1.739849,1.780000e-05,133.001270,0.007690,0.043,0.423,3.11400,0.22900,48.5,...,1,6119,165,4.444,0.056,1.031,0.341,286.50937,47.163219,14.757
6989,1,0.681402,2.430000e-06,132.181750,0.002850,0.147,0.309,0.86500,0.16200,103.6,...,1,6173,193,4.447,0.056,1.041,0.341,294.16489,47.176281,15.385


# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
target = df["koi_disposition"]
target.unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [42]:
df["koi_disposition"] = df["koi_disposition"].astype("category")
df["koi_disposition"]

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: category
Categories (3, object): [CANDIDATE, CONFIRMED, FALSE POSITIVE]

In [43]:
df["koi_disposition"].cat.codes

0       1
1       2
2       2
3       1
4       1
       ..
6986    2
6987    2
6988    0
6989    2
6990    2
Length: 6991, dtype: int8

In [44]:
df["koi_disposition"].cat.categories

Index(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype='object')

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

In [9]:
X_train.head()

,koi_fpflag_co,koi_period,koi_period_err1,koi_time0bk,koi_time0bk_err1,koi_impact,koi_impact_err1,koi_duration,koi_duration_err1,koi_depth,...,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_slogg,koi_slogg_err1,koi_srad,koi_srad_err1,ra,dec,koi_kepmag
6122,0,6.768901,7.380000e-05,133.077240,0.008440,0.150,0.305,3.61600,0.3060,123.1,...,1,5737,154,4.327,0.153,1.125,0.310,294.40472,39.351681,14.725
6370,0,0.733726,6.060000e-06,132.020050,0.007950,0.291,0.193,2.30900,0.2820,114.6,...,1,5855,158,4.578,0.033,0.797,0.211,284.50391,42.463860,15.770
2879,0,7.652707,6.540000e-05,134.460380,0.006190,0.970,0.879,79.89690,0.0000,641.1,...,1,6328,151,4.481,0.050,0.963,0.290,295.50211,38.983540,13.099
107,0,7.953547,1.910000e-05,174.662240,0.001820,0.300,0.145,2.63120,0.0595,875.4,...,1,4768,76,4.536,0.056,0.779,0.023,291.15878,40.750271,15.660
29,0,4.959319,5.150000e-07,172.258529,0.000083,0.831,0.016,2.22739,0.0075,9802.0,...,1,5712,77,4.359,0.110,1.082,0.173,292.16705,48.727589,15.263


In [10]:
y_train.head()

6122         CANDIDATE
6370    FALSE POSITIVE
2879    FALSE POSITIVE
107          CONFIRMED
29           CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [11]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(X_train_scaled.shape)
print(y_train.shape)                     

(5243, 27)
(5243,)


In [12]:
y_train[:5]

6122         CANDIDATE
6370    FALSE POSITIVE
2879    FALSE POSITIVE
107          CONFIRMED
29           CANDIDATE
Name: koi_disposition, dtype: object

In [13]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [14]:
encoded_y_train[:5]

array([0, 2, 2, 1, 0])

In [ ]:
# Step 2: Convert encoded labels to one-hot-encoding
# y_train_categorical = to_categorical(encoded_y_train)
# y_test_categorical = to_categorical(encoded_y_test)
# y_train_categorical

# Train the Model



In [15]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
model.fit(X_train_scaled, encoded_y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.7209612817089452
Testing Data Score: 0.7231121281464531


In [18]:
y_predict = model.predict(X_test_scaled)
y_predict[:5]

array([1, 0, 2, 2, 2])

In [29]:
df_predict = pd.DataFrame({
    "actual":encoded_y_test,
    "predicted":y_predict
})
df_predict

,actual,predicted
0,2,1
1,0,0
2,2,2
3,2,2
4,2,2
...,...,...
1743,0,2
1744,0,2
1745,1,1
1746,1,0


In [30]:
df_predict["actual"] = df_predict["actual"].astype("category")
df_predict["predicted"] = df_predict["predicted"].astype("category")
df_predict["actual"].cat.categories = ["Candidate", "Confirmed", "False Positive"]
df_predict["predicted"].cat.categories = ["Candidate", "Confirmed", "False Positive"]
df_predict.head()

,actual,predicted
0,False Positive,Confirmed
1,Candidate,Candidate
2,False Positive,False Positive
3,False Positive,False Positive
4,False Positive,False Positive


In [34]:
(df_predict["actual"] == df_predict["predicted"]).sum() / len(df_predict)

0.7231121281464531

In [28]:
df_predict["actual"].cat.categories = ["Candidate", "Confirmed", "False Positive"]
df_predict["actual"]

0       False Positive
1            Candidate
2       False Positive
3       False Positive
4       False Positive
             ...      
1743         Candidate
1744         Candidate
1745         Confirmed
1746         Confirmed
1747         Confirmed
Name: actual, Length: 1748, dtype: category
Categories (3, object): [Candidate, Confirmed, False Positive]

In [ ]:
# model.fit(X_train_scaled, y_train)

In [ ]:
# print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

In [ ]:
#????  Convert encoded labels to one-hot-encoding
# model.fit(X_train_scaled, y_train_categorical)
# print(f"Training Data Score: {model.score(X_train_scaled, y_train_categorical)}")
# print(f"Testing Data Score: {model.score(X_test_scaled, y_test_categorical)}") 

In [ ]:
model2 = SVC(kernel='rbf')

In [ ]:
model2.fit(X_train_scaled,encoded_y_train)

In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, encoded_y_test)}")

In [ ]:
model3 = SVC(kernel='sigmoid')
model3.fit(X_train_scaled,encoded_y_train)

In [ ]:
print(f"Training Data Score: {model3.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model3.score(X_test_scaled, encoded_y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],         #???? how to decide when is [1,5,10]
              'gamma': [0.0001, 0.001, 0.01, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
grid.score(X_test_scaled, encoded_y_test)

In [ ]:
predictions = grid.predict(X_test_scaled)
predictions

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions,
                            target_names=["CANDIDATE","FALSE POSITIVE","CONFIRMED"]))

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'GridSearch.sav'
joblib.dump(grid, filename)